# Lab11 工人智慧與自駕車(機率)
## Objectives

- 了解電腦影像的處理Pipeline
- 認識Duckiebot辨識路線的流程與方法
- 了解機率統計在實務中的實現與作法

## Tutorials

### Required(先讀完這份再看XD)

- [Part1 Lane Filter 作業說明]( https://www.youtube.com/watch?v=dEqg2c0jsRA)
- [Part2 Lane Filter 程式說明](https://www.youtube.com/watch?v=n3s6EvU65jw)
- [Part3 Lane Filter Debug補充](https://www.youtube.com/watch?v=b6Ztd52peoU)
- [Part4 Lane Filter Debug補充2](https://www.youtube.com/watch?v=emrbecL3Ffg)

### Optional

- [Introduction to computer vision](https://classroom.udacity.com/courses/ud810)
  - 可以挑選有興趣的Topics看

## Requirements:

### 安裝需要的新packages(在自己電腦上的python)

```
pip install scipy matplotlib
```

### 下載Lab11的作業Code到自己的電腦中

```
git clone -b Lab11 https://github.com/GrassLab/NCTU_CS_ZOO.git Lab11
或是手動選擇Lab11的分支下載zip檔
```

之後所有Steps都已經假設你已經SSH連進去小鴨車，並運行好```python -m nodes.duckiebot```

(和HW10/Challenge10一樣，開好就可以不管，且**小鴨車上面跑Lab10已經放過的code即可**，可以不用重傳Lab11的上去)

- **複製原本在Lab10使用的** ```Lab10/duckiebot_cs_zoo/app/data/camera_calibration/camera_data.pkl```到```Lab11/duckiebot_cs_zoo/app/data/camera_calibration``` 中 **(請自己創建資料夾)**

#### 作業相關重要檔案，標記HW Step代表你依序需要開啟(看懂並稍做修改)的部分

- duckiebot_cs_zoo/app/lane 裡面的
  - lane_detector_demo.py: **HW Step1**
  - lane_filter_lr_demo.py: **HW Step3**
  - lane_filter_stop_demo.py: **HW Step4**
- duckiebot_cs_zoo/app裡面的
  - automatic_parking_color.py: **HW Step6**
- duckiebot_cs_zoo/libs裡面的
  - lane_detector.py: **HW Step2**
  - lane_filter_lr.py: Reference for Step3/4
  - lane_filter_stop.py: **HW Step5**

#### (Optional)其他方便你用的工具

- Camera Calibration系列遠端版(就是你duckiebot server打開後，直接從你電腦上的python就能做校正了):位於```duckiebot_cs_zoo/app/remote_calib```下面
  
  - 跑的時候記得，因為有用到DuckiebotClient，環境變數記得設定好(DUCKIE還有PORT)
  
  - **範例用法(如果你有要重校正才需要跑)**
  
    ```bash
    python -m app.remote_calib.remote_collect_calibration_image
    python -m app.remote_calib.remote_camera_calibration
    python -m app.remote_calib.remote_extrinsic_calibration_pose
    python -m app.remote_calib.remote_verify_calibration
    ```
  
- ```duckiebot_cs_zoo/utils/video.py```方便你做錄影還有Replay，例如可以先路一段車車行走的影片，再Replay看到的照片，就不用一直去現場跑，用法在程式裡面，可自行理解服用(非作業要求)，最後一次lab也許會用到XD

### 停車要求

- 這次和HW10不同，**小鴨車初始位置請直接正對著地圖，在停車十字下方約25公分開始**，只要能維持直線前進然後再靠近停車十字時停住即可，輪胎只要都在框內即算完成，角度45度以內，**不用寫得太完美，可以停下即可**

- 建議可以先使用remote_client.py 裡面的 st指令找到trim的參數，可以有效幫助走直線更好

  ```python
  python -m app.remote_client
  Cmd:kb
  然後讓小鴨車前進兩公尺左右(w前進，2公尺後空白鍵停止後按q)
  Cmd:st
  Input trim val: 0.2
  Cmd:kb
  然後讓小鴨車前進兩公尺左右(w前進，2公尺後空白鍵停止後按q)
  Cmd:st
  Input trim val: -0.2
  Cmd:kb
  然後讓小鴨車前進兩公尺左右(w前進，2公尺後空白鍵停止後按q)
  .....
  
  trim的值先嘗試+-0.2,你會發現車子偏左或偏右,然後可以增加或是減少trim的數值與正負,讓車子兩公尺前進後,幾乎是走直線,即可記下來現在的trim值,之後都記得用
  # duckiebot = DuckiebotClient()
  duckiebot.set_trim(0.01)
  來設定這個參數
  ```

- 可以嘗試讓自己的omega在設定時，與error成比例，例如，現在要走0度，但現在車車角度實際是60度，那omega可以用下列來處理

  ```python
  import numpy as np
  P=20
  OMEGA_MAX=2
  target=0 # degree
  phi, d = lane_filter_lr.get_estimate()
  angle=np.rad2deg(phi) # 假設angle=60度，代表角度往左偏很多
  error=target-angle # -60
  omega=np.clip(error/P,-OMEGA_MAX,OMEGA_MAX) # omega=-2
  ```

  之後只需要控制P，來讓車車根據誤差成比例地調整旋轉速度，OMEGA_MAX則是限制最大絕對值避免走太偏

## 實作流程(建議先讀完，開啟一下code再開始看TA教學影片跟著操作)
所有Steps都已經假設你SSH進去好小鴨車後```cd Lab10/duckiebot_cs_zoo```，並運行好```python -m nodes.duckiebot```(和HW10/Challenge10一樣，開好就可以不管，小鴨車上面跑Lab10已經放過的code即可)

**建議將小鴨車攝影機可以看到的環境用厚紙板或貼一些白紙遮一下背景，避免雜物顏色影響路線判斷**

1. HW Step1: 你需要完成**顏色校正與線段偵測**的操作，請參考教學影片進行操作
   
   - 記得你的```COLOR_RANGE_FILE```名稱，之後會經常用到
   - Challenge11和12你都會需要現場進行調整，尤其是**tune_color_ranges**會需要根據現場光源微調，因此請熟悉這部分操作
   
   - 你需要跑的指令(**強烈建議還是如影片所示搭配debugger/IDE來操作，若花了很多時間還是不會自己設定VS Code或是Pycharm的Debugger可以寄信約TA**)
   
     ```bash
     cd duckiebot_cs_zoo
     python -m app.lane.lane_detector_demo
     ```
   
2. HW Step2: 參考```lane_detector.py```，將 Step1調整出來的參數依序填進去detect_lane函數中，**如果Step1有重做，這步也記得重做**

   - 你需要編輯的檔案```duckiebot_cs_zoo/libs/lane_detector.py```
     - 將你在```duckiebot_cs_zoo/app/lane/lane_detector_demo.py```的決定好的參數，填入到```duckiebot_cs_zoo/libs/lane_detector.py```裡面42行開始的```detect_lane```function中

3. HW Step3: 參考```lane_filter_lr_demo.py```，你只需要填入正確的```COLOR_RANGE_FILE```就能跑起來，可以觀察吐出來的$d$ 和 $\phi$，並可以手動開啟與關閉結果顯示

   - 你需要編輯的檔案```duckiebot_cs_zoo/app/lane/lane_filter_lr_demo.py```
     
     - 填入正確的```COLOR_RANGE_FILE```
     
   - 你需要跑的指令

     ```bash
     python -m app.lane.lane_filter_lr_demo
     ```

4. HW Step4: 和Step3相似，開啟```lane_filter_stop_demo.py```後只需填入正確的```COLOR_RANGE_FILE```就能跑起來，但吐出來的```Estimate: dist```都會是一個很怪的定值(因為你還沒開始寫到codeXD)
   - 你需要編輯的檔案```duckiebot_cs_zoo/app/lane/lane_filter_stop_demo.py```
     
     - 填入正確的```COLOR_RANGE_FILE```
     
   - 你需要跑的指令
   
     ```bash
     python -m app.lane.lane_filter_stop_demo
     ```

5. HW Step5: 你要算一下**高一數學**還有實作```lane_filter_stop.py```裡面有被

   ```python
   #Start >>> Your code here
   Write You Code Here
   #End <<< Your code here
   ```

   包起來的部分，其他地方勿動，共有四處，請參考```Lab11_notes.pdf```

   當你完成後，可以再跑一次Step4的```lane_filter_stop_demo.py```，確定跑起來數值合理

   理論上只會需要用到```numpy```的數學運算，建議先推導好再實作

   - 你需要編輯的檔案```duckiebot_cs_zoo/libs/lane_filter_stop.py```

     - 參考```Lab11_notes.pdf```，總共有四處

   - 你需要跑的指令(改完```lane_filter_stop.py```後)

     ```bash
     python -m app.lane.lane_filter_stop_demo
     ```

6. HW Step6: 填入正確的```COLOR_RANGE_FILE```，然後完成```automatic_parking_color.py```，和Lab10 HW一樣，只要大概停到框框中即可，可以思考一下，你有$d,\phi,\text{dist}$後，是否其實就是另一個座標系呢？所以停車邏輯基本上和 Lab10 HW是相同的處理方法

   - 你需要編輯的檔案```duckiebot_cs_zoo/app/automatic_parking_color.py```

     - 填入正確的```COLOR_RANGE_FILE```
     - 確定好$d,\phi,\text{dist}$都能正確拿到合理數值後，實作直線開進去停車的邏輯

   - 你需要跑的指令

     ```bash
     python -m app.automatic_parking_color
     ```

## HINTS

Normal Vector，符號記成$\vec{n}$，在程式中我們都已經Normalize過，所以他的長度都會是1

Mid其實等價於一個從原點出發的向量，記成$\vec{m}$

已知 $\vec{n}$和$\vec{m}$的夾角$cos(\theta)=\frac{\vec{n}\cdot\vec{m}}{\|\vec{n}\|\cdot\|\vec{m}\|}=\frac{\vec{n}\cdot\vec{m}}{\|\vec{m}\|}$

我們要求的長度(**線到達車子的距離**，假設是bottom case)即是 $\|m\|\cdot cos(\theta)$

但要再記得換算成停車十字到達車子的距離，且正負號符合規定XD